# Simulate Data


**Here is only a trip of SINGLE FieldGrn Data**

In [2]:
from fieldnn.utils.layerfn import traverse
from fieldnn.utils.simulate import get_next_info, get_simulated_tensor_from_fldname

# Holder

1. Current Layer:
```python
leng_mask = info_idx == 0 # or info_idx != 0
leng = leng_mask.sum(-1)
```

2. Transfer

```python
old_leng = leng
```


3. Next Layer
```python
leng_mask = old_leng != 0
leng = (leng_mask == 0).sum(-1)
```

In [3]:
from fieldnn.utils.layerfn import get_Layer2Holder, align_psn_idx

In [4]:
import torch
# ======= within forward

###############
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 5001
Ignore_PSN_Layers = ['B', 'P']
###############


fullname = 'B-PatRec:EC-ECRec:Diag-DiagRec:DiagV-DiagVdftGrn'
info_idx = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(info_idx.shape)
holder = torch.LongTensor(info_idx)


############### gsn_embeddings
Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
name = fullname.split('-')[-1]
Layer2Holder = get_Layer2Holder(fullname, holder, Ignore_PSN_Layers)
psn_layers = list(reversed([i for i in Layer2Idx if i not in Ignore_PSN_Layers]))


print(Layer2Idx)
print(name)
print([i for i in Layer2Holder], '<---- Layer2Holder')
print(psn_layers, '<---- psn_layers')

for source_layer in psn_layers:
    cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
    print(source_layer, cpsn_idx.shape)

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 4)
3
3 --> (3, 6, 4)
4 --> (3, 6, 4, 1)
(3, 6, 4, 1)
{'B': 0, 'PatRec:EC': 1, 'ECRec:Diag': 2, 'DiagRec:DiagV': 3, 'DiagVdftGrn': 4}
DiagVdftGrn
['DiagVdftGrn', 'DiagRec:DiagV', 'ECRec:Diag', 'PatRec:EC'] <---- Layer2Holder
['DiagVdftGrn', 'DiagRec:DiagV', 'ECRec:Diag', 'PatRec:EC'] <---- psn_layers
DiagVdftGrn torch.Size([3, 6, 4, 1])
DiagRec:DiagV torch.Size([3, 6, 4, 1])
ECRec:Diag torch.Size([3, 6, 4, 1])
PatRec:EC torch.Size([3, 6, 4, 1])


# Expander

## Module

In [6]:
import torch
import torch.nn.functional as F

from fieldnn.nn.embedding import EmbeddingLayer
from fieldnn.nn.lmembed import LMEmbedLayer
from fieldnn.utils.layerfn import get_Layer2Holder, align_psn_idx

# from ..nn.embedding import EmbeddingLayer
# from ..nn.lmembed import LMEmbedLayer
# from ..utils.layer import get_Layer2Holder, align_psn_idx


class Expander_Layer(torch.nn.Module):
    
    '''Only for Increasing Embedding Dimensions'''
    def __init__(self, input_fullname, output_fullname, expander_layer_para):
        super(Expander_Layer, self).__init__()
        
        # tensor.shape[-1]
        self.input_size = expander_layer_para['input_size']
        self.output_size = expander_layer_para['output_size']
        self.Ignore_PSN_Layers = expander_layer_para['Ignore_PSN_Layers']
        
        # Part 1: embedding
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        
        assert 'Grn' == input_fullname[-3:]
        assert input_fullname.replace('Grn', '') == output_fullname
        
        nn_name, para = expander_layer_para[input_fullname]
        
        if nn_name.lower() == 'embedding':
            self.Embed = EmbeddingLayer(**para)
            self.embed_size = para['embedding_size']
        elif nn_name.lower() == 'lmembed':
            # TODO
            self.Embed = LMEmbedLayer(**para)
            self.embed_size = para['embedding_size']
        else:
            raise ValueError(f'NN "{nn_name}" is not available')
            
        assert self.embed_size == self.output_size
        
        # Part 2: PSN embedding
        # psn_layers = expander_layer_para['psn_layers']
        # self.PSN_Embed_Dict = torch.nn.ModuleDict()
        # for layername in psn_layers:
        #     para = generate_psn_embed_para(layername, self.embed_size)
        #     self.PSN_Embed_Dict[layername] = EmbeddingLayer(**para)
        
        # Part 3: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in expander_layer_para['postprocess'].items():
            if method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.embed_size, **config)
                
    # def get_psn_embed(self, fullname, holder):
    #     name = fullname.split('-')[-1]
    #     Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
    #     Layer2Holder = get_Layer2Holder(fullname, holder, self.Ignore_PSN_Layers)
        
    #     psn_embed = 0
    #     for source_layer, Embed in self.PSN_Embed_Dict.items():
    #         cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
    #         psn_embed = psn_embed + Embed(cpsn_idx)
        
    #     return psn_embed

    def forward(self, fullname, holder, info = 'Empty'):
        '''
            fullname: full name of field, GRN is here
            holder: info_idx
            info: info_wgt
        '''
        assert self.input_fullname == fullname
        leng_mask = holder == 0
        embed = self.Embed(holder)
        
        if type(info) != str:
            embed = embed * info.unsqueeze(-1)
        
        # Comments: move these to Learner.
        # if len(self.PSN_Embed_Dict):
        #     psn_embed = self.get_psn_embed(fullname, holder)
        #     embed = embed + psn_embed
        
        for nn, layer in self.postprocess.items():
            embed = layer(embed)
        
        return self.output_fullname, holder, embed

In [7]:
# from fieldnn.sublayer.expander import Expander_Layer

## Config

In [8]:
# def get_expander_para(fullname, 
#                       nn_name, nn_para, 
#                       embed_size, 
#                       Ignore_PSN_Layers,
#                       postprocess
#                      ):
    
#     expander_layer_para = {}
#     #(0) Input size, output size
#     expander_layer_para['input_size'] = None
#     expander_layer_para['output_size'] = embed_size

    
#     #(1) Main NN
#     if nn_name.lower() == 'embedding':
#         para = {'embedding_size': embed_size,
#                 'init': 'random', 
#                 'input_size': 5000} # will be updated in input_size
#         # print(nnpara)
#         assert 'input_size' in nn_para
#         for k, v in nn_para.items():
#             if k in para: para[k] = v
#     else:
#         raise ValueError(f'The NN "{nn_name}" is not available yet')
#     expander_layer_para[fullname] = nn_name, para
    
#     # (2) PSN Embed
#     # name = fullname.split('-')[-1]
#     # Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
#     # psn_layers = list(reversed([i for i in Layer2Idx if i not in Ignore_PSN_Layers]))
#     # expander_layer_para['psn_layers'] = psn_layers
    
    
#     # (3) Post Process
#     expander_layer_para['postprocess'] = postprocess
    
#     expander_layer_para['Ignore_PSN_Layers'] = Ignore_PSN_Layers
    
#     return expander_layer_para

In [9]:
from fieldnn.utils.parafn import get_expander_para

## Usage

In [10]:
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 5001

Ignore_PSN_Layers = ['B', 'P']

fullname = 'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn'
# fld_name = 'B2St2TkGrn'
info_idx = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(info_idx.shape)
holder = torch.LongTensor(info_idx)

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 7)
3
3 --> (3, 6, 7)
4 --> (3, 6, 7, 8)
(3, 6, 7, 8)


In [11]:
embed_size = 512

process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}


nn_name = 'Embedding'
nn_para = {'input_size': vocab_size}
postprocess = process
Ignore_PSN_Layers = Ignore_PSN_Layers


input_fullname = fullname
output_fullname = fullname.replace('Grn', '')
print(input_fullname)
print(output_fullname)
expander_layer_para = get_expander_para(input_fullname, nn_name, nn_para, embed_size, 
                                        Ignore_PSN_Layers, 
                                        postprocess
                                       )
expander_layer_para

B-P-EC:Diag-Rec:DiagV-DiagVdftGrn
B-P-EC:Diag-Rec:DiagV-DiagVdft


{'input_size': None,
 'output_size': 512,
 'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn': ('Embedding',
  {'embedding_size': 512, 'init': 'random', 'input_size': 5001}),
 'postprocess': {'activator': 'gelu',
  'dropout': {'p': 0.5, 'inplace': False},
  'layernorm': {'eps': 1e-05, 'elementwise_affine': True}},
 'Ignore_PSN_Layers': ['B', 'P']}

In [12]:
Expander = Expander_Layer(input_fullname, output_fullname, expander_layer_para)
Expander

Expander_Layer(
  (Embed): EmbeddingLayer(
    (embedding): Embedding(5001, 512, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [13]:
info_wgt = 'None'
fullname, holder, info = Expander(fullname, holder, info_wgt)
print(info.shape)
print(fullname)
print(holder.shape)

torch.Size([3, 6, 7, 8, 512])
B-P-EC:Diag-Rec:DiagV-DiagVdft
torch.Size([3, 6, 7, 8])


# Learner

## Module

In [15]:
import os
import torch
import numpy as np

# from ..nn.rnn import RNNLayer
# from ..nn.cnn import CNNLayer
# from ..nn.tfm import TFMLayer
# from ..nn.linear import LinearLayer
# from ..nn.embedding import EmbeddingLayer
# from ..utils.layerfn import orderSeq, restoreSeq, align_psn_idx, get_Layer2Holder
# from ..utils.parafn import generate_psn_embed_para

from fieldnn.nn.rnn import RNNLayer
from fieldnn.nn.cnn import CNNLayer
from fieldnn.nn.tfm import TFMLayer
from fieldnn.nn.linear import LinearLayer
from fieldnn.nn.embedding import EmbeddingLayer
from fieldnn.utils.layerfn import orderSeq, restoreSeq, align_psn_idx, get_Layer2Holder
from fieldnn.utils.parafn import generate_psn_embed_para


class Learner_Layer(torch.nn.Module):
    def __init__(self, input_fullname, output_fullname, learner_layer_para):
        super(Learner_Layer, self).__init__()
        
        # Part 0: Meta
        self.fullname = input_fullname
        
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        assert self.input_fullname == self.output_fullname
        
        self.input_size = learner_layer_para['input_size']
        self.output_size = learner_layer_para['output_size']
        self.embed_size = self.input_size
        
        # Part 1: NN
        nn_name, para = learner_layer_para[self.fullname]
        if nn_name.lower() == 'cnn':
            self.Learner = CNNLayer(**para)
        elif nn_name.lower() == 'rnn':
            self.Learner = RNNLayer(**para)
        elif nn_name.lower() == 'tfm':
            assert self.input_size == self.output_size
            self.Learner = TFMLayer(**para)
        elif nn_name.lower() == 'linear':
            self.Learner = LinearLayer(**para)
        else:
            raise ValueError(f'NN "{nn_name}" is not available')

        # Part 2: PSN embedding
        psn_layers = learner_layer_para['psn_layers']
        self.PSN_Embed_Dict = torch.nn.ModuleDict()
        for layername in psn_layers:
            para = generate_psn_embed_para(layername, self.embed_size)
            self.PSN_Embed_Dict[layername] = EmbeddingLayer(**para)
            
        # Part 2: EmbedProcess
        self.embedprocess = torch.nn.ModuleDict()
        for method, config in learner_layer_para['embedprocess'].items():
            if method == 'dropout':
                self.embedprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.embedprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
        
        # Part 3: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in learner_layer_para['postprocess'].items():
            if method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)

        self.Ignore_PSN_Layers = learner_layer_para['Ignore_PSN_Layers']
        
    def get_psn_embed(self, fullname, holder):
        name = fullname.split('-')[-1]
        Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
        Layer2Holder = get_Layer2Holder(fullname, holder, self.Ignore_PSN_Layers)
        
        psn_embed = 0
        for source_layer, Embed in self.PSN_Embed_Dict.items():
            cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
            psn_embed = psn_embed + Embed(cpsn_idx)
        return psn_embed
    
    def reshape(self, info, leng_mask):
        nbs = np.array(info.shape[:-2]).prod()
        ngrn, dim = info.shape[-2:]
        # print(nbs, ngrn, dim)
        
        tmp_info = info.contiguous().view(nbs, ngrn, dim)
        # print(tmp_info.shape)

        tmp_leng_mask = leng_mask.contiguous().view(nbs, ngrn)
        # print(tmp_leng_mask.shape)

        tmp_leng = (tmp_leng_mask == 0).sum(-1)
        # print(tmp_leng.shape)
        
        ord_info,      ord_leng, r_idx = orderSeq(tmp_info, tmp_leng)
        ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
        return ord_info, ord_leng_mask, r_idx
    
    def restore(self, ord_info_output, leng_mask, r_idx):
        info_new = restoreSeq(ord_info_output, r_idx)
        output_size = info_new.shape[-1]
        info_output = info_new.view(*list(leng_mask.shape) + [output_size])
        return info_output
        
    def forward(self, fullname, holder, info):
        assert self.input_fullname == fullname
        
        # TODO: this step needs close review. 
        # (1) adding psn embed
        if len(self.PSN_Embed_Dict):
            psn_embed = self.get_psn_embed(fullname, holder)
            for nn, layer in self.embedprocess.items():
                psn_embed = layer(psn_embed)
            info = info + psn_embed
            # TODO: process psn_embed? Do we need the further embed process?
            
        # (2) do the calculation
        leng_mask = holder == 0
        ord_info, ord_leng_mask, r_ix = self.reshape(info, leng_mask)
        ord_info_output = self.Learner(ord_info, ord_leng_mask)
        info = self.restore(ord_info_output, leng_mask, r_ix)
        
        # (3) post-process
        for nn_name, layer in self.postprocess.items():
            info = layer(info)
            
        # we do not change the fullname and holder
        return self.output_fullname, holder, info

In [16]:
# from fieldnn.sublayer.learner import Learner_Layer

## Config

In [17]:
# # from fieldnn.utils.parafn import generate_psn_embed_para

# def get_learner_para(fullname, 
#                      nn_name, nn_para, 
#                      input_size, output_size, 
#                      Ignore_PSN_Layers, 
#                      embedprocess,
#                      postprocess):
#     learner_layer_para = {}
#     # (0) Size
#     learner_layer_para['input_size'] = input_size
#     learner_layer_para['output_size'] = output_size
    
    
#     # (1) NN
#     if nn_name.lower() == 'tfm':
#         assert input_size == output_size
#         para =  {'input_size': output_size,
#                  'output_size': output_size,
#                  'nhead': 8,
#                  'num_encoder_layers': 6,
#                  'num_decoder_layers': 0,
#                  'dim_feedforward': 2048,
#                  'tfm_dropout': 0.1,
#                  'tfm_activation': 'relu'}
#         for k, v in nn_para.items():
#             if k in para: para[k] = v
            
#     else:
#         raise ValueError(f'The NN "{nn_name}" is not available yet')
    
#     learner_layer_para[fullname] = nn_name, para
    
#     # (2) PSN Embed
#     name = fullname.split('-')[-1]
#     Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
#     psn_layers = list(reversed([i for i in Layer2Idx if i not in Ignore_PSN_Layers]))
#     learner_layer_para['psn_layers'] = psn_layers
#     learner_layer_para['embedprocess'] = embedprocess
#     learner_layer_para['Ignore_PSN_Layers'] = Ignore_PSN_Layers
    
#     # (3) Post Process
#     learner_layer_para['postprocess'] = postprocess

#     return learner_layer_para

In [18]:
from fieldnn.utils.parafn import get_learner_para

## Usage

In [19]:
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 100

Ignore_PSN_Layers = ['B', 'P']

fullname = 'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn'
# fld_name = 'B2St2TkGrn'
info_idx = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(info_idx.shape)
holder = torch.LongTensor(info_idx)

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 9)
3
3 --> (3, 6, 9)
4 --> (3, 6, 9, 2)
(3, 6, 9, 2)


In [20]:
from pprint import pprint
# embed_size = 512
vocab_size = 5001
process = {
    'dropout' :{'p':0.5, 'inplace':False},
    'layernorm': {'eps': 1e-05, "elementwise_affine":True},
}


###########
nn_name = 'Embedding'
nn_para = {'input_size': vocab_size}
postprocess = process
###########

input_fullname = fullname
output_fullname = input_fullname.replace('Grn', '')

expander_layer_para = get_expander_para(fullname, nn_name, nn_para, embed_size, 
                                        Ignore_PSN_Layers, 
                                        postprocess
                                       )
pprint(expander_layer_para)
Expander = Expander_Layer(input_fullname, output_fullname, expander_layer_para)
Expander

{'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn': ('Embedding',
                                       {'embedding_size': 512,
                                        'init': 'random',
                                        'input_size': 5001}),
 'Ignore_PSN_Layers': ['B', 'P'],
 'input_size': None,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
                 'layernorm': {'elementwise_affine': True, 'eps': 1e-05}}}


Expander_Layer(
  (Embed): EmbeddingLayer(
    (embedding): Embedding(5001, 512, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [21]:
info = 'None'
fullname, holder, info = Expander(fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)

torch.Size([3, 6, 9, 2, 512])
B-P-EC:Diag-Rec:DiagV-DiagVdft
torch.Size([3, 6, 9, 2])


In [22]:
###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########
input_fullname = output_fullname
output_fullname = input_fullname
learner_layer_para  = get_learner_para(fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
pprint(learner_layer_para)

Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)
Learner

{'B-P-EC:Diag-Rec:DiagV-DiagVdft': ('TFM',
                                    {'dim_feedforward': 2048,
                                     'input_size': 512,
                                     'nhead': 8,
                                     'num_decoder_layers': 0,
                                     'num_encoder_layers': 6,
                                     'output_size': 512,
                                     'tfm_activation': 'relu',
                                     'tfm_dropout': 0.1}),
 'Ignore_PSN_Layers': ['B', 'P'],
 'embedprocess': {'dropout': {'inplace': False, 'p': 0.5},
                  'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},
 'input_size': 512,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
                 'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},
 'psn_layers': ['DiagVdft', 'Rec:DiagV', 'EC:Diag']}


Learner_Layer(
  (Learner): TFMLayer(
    (transformer): Transformer(
      (encoder): TransformerEncoder(
        (layers): ModuleList(
          (0-5): 6 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
            )
            (linear1): Linear(in_features=512, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=512, bias=True)
            (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
        (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (decoder): TransformerDecoder(
        (layers): ModuleList()
        (norm): Layer

In [23]:
fullname, holder, info = Learner(fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)

torch.Size([3, 6, 9, 2, 512])
B-P-EC:Diag-Rec:DiagV-DiagVdft
torch.Size([3, 6, 9, 2])


In [24]:
info.shape[:-2]

torch.Size([3, 6, 9])

# Merger

## Module

In [25]:
import torch
from fieldnn.nn.op import MergerLayer
# from ..nn.op import MergerLayer

class Merger_Layer(torch.nn.Module):
    def __init__(self, input_fullname, output_fullname, merger_layer_para):
        super(Merger_Layer, self).__init__()
        
        # n * (bs, .., c_inp) --> (bs, ..., n, c_inp)
        # Meta Info
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        
        # Part 0: sizes
        self.input_size = merger_layer_para['input_size']
        self.output_size = merger_layer_para['output_size']
        
        # Part 1: NN
        nn_name, nn_para = merger_layer_para[input_fullname]
        if nn_name.lower() == 'merger':
            self.Merger = MergerLayer()
        else:
            raise ValueError(f'The NN "{nn_name}" is not available')
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in merger_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
                
    def forward(self, input_fullname, fullname2data):
        
        fld_list = input_fullname.split('^')
        assert len(fld_list) == len(fullname2data)
        # (1) holder
        holder = self.Merger([data['holder'] for fld, data in fullname2data.items()], -1)
        
        # (2) merge data
        info = self.Merger([data['info'] for fld, data in fullname2data.items()], -2)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)
        
        # (4) names
        # prefix = ['-'.join(i.split('-')[:-1]) for i in fullname2data][0]
        # assert prefix == self.fieldname
        # fullname_new = prefix + '2GrnRec2FldType'
        
        return self.output_fullname, holder, info

In [26]:
# from fieldnn.sublayer.merger import Merger_Layer

## Config

In [27]:
# def get_merger_para(fullname, nn_name, nn_para, input_size, output_size, postprocess):
#     merger_layer_para = {}
    
#     merger_layer_para[fullname] = nn_name, nn_para
    
#     merger_layer_para['input_size'] = input_size
#     merger_layer_para['output_size'] = output_size

#     merger_layer_para['postprocess'] = postprocess
    
#     return merger_layer_para

In [28]:
from fieldnn.utils.parafn import get_merger_para

## Usage

In [29]:
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 100

Ignore_PSN_Layers = ['B', 'P']

fullname = 'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn'
# fld_name = 'B2St2TkGrn'
info_idx = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(info_idx.shape)
holder = torch.LongTensor(info_idx)

from pprint import pprint
# embed_size = 512
vocab_size = 5001
process = {
    'dropout' :{'p':0.5, 'inplace':False},
    'layernorm': {'eps': 1e-05, "elementwise_affine":True},
}


###########
nn_name = 'Embedding'
nn_para = {'input_size': vocab_size}
postprocess = process
###########

input_fullname = fullname
output_fullname = input_fullname.replace('Grn', '')

expander_layer_para = get_expander_para(fullname, nn_name, nn_para, embed_size, 
                                        Ignore_PSN_Layers, 
                                        postprocess
                                       )
pprint(expander_layer_para)
Expander = Expander_Layer(input_fullname, output_fullname, expander_layer_para)
print(Expander)


info = 'None'
fullname, holder, info = Expander(fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)




1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 5)
3
3 --> (3, 6, 5)
4 --> (3, 6, 5, 2)
(3, 6, 5, 2)
{'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn': ('Embedding',
                                       {'embedding_size': 512,
                                        'init': 'random',
                                        'input_size': 5001}),
 'Ignore_PSN_Layers': ['B', 'P'],
 'input_size': None,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
                 'layernorm': {'elementwise_affine': True, 'eps': 1e-05}}}
Expander_Layer(
  (Embed): EmbeddingLayer(
    (embedding): Embedding(5001, 512, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)
torch.Size([3, 6, 5, 2, 512])
B-P-EC:Diag-Rec:DiagV-DiagVdft
torch.Size([3, 6, 5, 2])


In [30]:
###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########
input_fullname = output_fullname
output_fullname = input_fullname
learner_layer_para  = get_learner_para(fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
pprint(learner_layer_para)

Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)
# print(Learner)

fullname, holder, info = Learner(fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)

{'B-P-EC:Diag-Rec:DiagV-DiagVdft': ('TFM',
                                    {'dim_feedforward': 2048,
                                     'input_size': 512,
                                     'nhead': 8,
                                     'num_decoder_layers': 0,
                                     'num_encoder_layers': 6,
                                     'output_size': 512,
                                     'tfm_activation': 'relu',
                                     'tfm_dropout': 0.1}),
 'Ignore_PSN_Layers': ['B', 'P'],
 'embedprocess': {'dropout': {'inplace': False, 'p': 0.5},
                  'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},
 'input_size': 512,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
                 'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},
 'psn_layers': ['DiagVdft', 'Rec:DiagV', 'EC:Diag']}
torch.Size([3, 6, 5, 2, 512])
B-P-EC:Diag-Rec:DiagV-DiagVdft
torch.Size([3, 6, 5, 2])


In [31]:
# def get_fullname_from_inputs(fullname_list):
#     names = [i.split('-')[-1] for i in fullname_list]
    
#     prefix = ['-'.join(i.split('-')[:-1]) for i in fullname_list][0]
    
#     table_row_indicator = list(set([i.count(':') for i in names]))[0]
    
#     if table_row_indicator == 0:
#         fullname = f'{prefix}-{"&".join(names)}'
#     elif table_row_indicator == 1:
#         table_name = [i.split(':')[0] for i in names][0]
#         columns =  [i.split(':')[-1] for i in names]
#         fullname = f'{prefix}-{table_name}-{"&".join(columns)}'
        
#     return fullname

In [33]:
from fieldnn.utils.parafn import get_fullname_from_inputs

## option 1
# fullnamea = 'B-P-EC:Diag-Rec:DiagV-DiagVdftA'
# fullnameb = 'B-P-EC:Diag-Rec:DiagV-DiagVdftB'

## option 2
fullnamea = 'B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA'
fullnameb = 'B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB'

fullname_list = [fullnamea, fullnameb]
fullname2data = {i: {'holder': holder, 'info': info} for i in fullname_list}

input_fullname = '^'.join(fullname_list)
print(input_fullname)
output_fullname = get_fullname_from_inputs(fullname_list)
print(output_fullname)

B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA^B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB
B-P-EC:Diag-Rec:DiagV-VRec-DiagVdftA&DiagVdftB


In [34]:
for fn_input, data in fullname2data.items():
    print(fn_input, data['holder'].shape, data['info'].shape)
print(fullname_list)

B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA torch.Size([3, 6, 5, 2]) torch.Size([3, 6, 5, 2, 512])
B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB torch.Size([3, 6, 5, 2]) torch.Size([3, 6, 5, 2, 512])
['B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA', 'B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB']


In [35]:
# fullname_list # in this case, the two fullnames are not the same: 

In [36]:
#########
nn_name = 'Merger'
nn_para = {}
input_size = embed_size
output_size = embed_size
postprocess = process
#########

merger_layer_para = get_merger_para(input_fullname, nn_name, nn_para, input_size, output_size, postprocess)
merger_layer_para

{'B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftA^B-P-EC:Diag-Rec:DiagV-VRec:DiagVdftB': ('Merger',
  {}),
 'input_size': 512,
 'output_size': 512,
 'postprocess': {'dropout': {'p': 0.5, 'inplace': False},
  'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}

In [37]:
Merger = Merger_Layer(input_fullname, output_fullname, merger_layer_para)
Merger

Merger_Layer(
  (Merger): MergerLayer()
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [38]:
output_fullname, holder, info = Merger(input_fullname, fullname2data)
print(holder.shape)
print(info.shape)
print(output_fullname)

torch.Size([3, 6, 5, 2, 2])
torch.Size([3, 6, 5, 2, 2, 512])
B-P-EC:Diag-Rec:DiagV-VRec-DiagVdftA&DiagVdftB


In [39]:
info

tensor([[[[[[ 1.1960e-02,  1.1960e-02,  4.4601e-01,  ...,  1.1960e-02,
             -5.3202e-03,  1.1960e-02],
            [-7.0848e-03, -7.0848e-03, -7.0848e-03,  ..., -7.0848e-03,
             -7.0848e-03, -7.0848e-03]],

           [[ 6.5475e-02,  6.5475e-02,  6.0348e-02,  ...,  6.0348e-02,
              6.0348e-02,  7.3955e-02],
            [-6.5196e-02, -6.0204e-02, -6.5196e-02,  ..., -6.5196e-02,
             -6.5196e-02, -6.5196e-02]]],


          [[[ 1.4748e-02,  1.4748e-02,  1.4748e-02,  ..., -4.3735e-02,
             -4.3735e-02, -4.3735e-02],
            [ 2.6660e-02,  6.4197e-03,  7.2832e-03,  ...,  6.4197e-03,
              7.1186e-02,  6.4197e-03]],

           [[-2.5645e-02, -5.3555e-01, -1.0653e+00,  ...,  2.3217e+00,
              5.5946e-02,  2.2144e-01],
            [ 2.6404e-01,  9.9920e-03, -1.0590e+00,  ...,  2.4235e+00,
              9.9920e-03,  2.6404e-01]]],


          [[[-2.3705e-02, -5.6191e-02,  5.1534e-01,  ..., -2.3705e-02,
             -2.3705e-02, -2.

In [40]:
holder

tensor([[[[[20, 20],
           [18, 18]],

          [[ 1,  1],
           [24, 24]],

          [[30, 30],
           [ 0,  0]],

          [[95, 95],
           [88, 88]],

          [[10, 10],
           [ 0,  0]]],


         [[[ 3,  3],
           [87, 87]],

          [[97, 97],
           [ 0,  0]],

          [[37, 37],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]]],


         [[[10, 10],
           [92, 92]],

          [[43, 43],
           [ 0,  0]],

          [[88, 88],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]]],


         [[[41, 41],
           [ 0,  0]],

          [[38, 38],
           [68, 68]],

          [[10, 10],
           [55, 55]],

          [[ 0,  0],
           [ 0,  0]],

          [[ 0,  0],
           [ 0,  0]]],


         [[[45, 45],
           [ 0,  0]],

          [[94, 94],
           [93, 93]],

          [[92, 92],
   

In [41]:
print(output_fullname)

B-P-EC:Diag-Rec:DiagV-VRec-DiagVdftA&DiagVdftB


In [42]:
###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########


input_fullname = output_fullname
output_fullname = input_fullname
learner_layer_para  = get_learner_para(input_fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
pprint(learner_layer_para)

Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)
# print(Learner)

fullname, holder, info = Learner(input_fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)

{'B-P-EC:Diag-Rec:DiagV-VRec-DiagVdftA&DiagVdftB': ('TFM',
                                                    {'dim_feedforward': 2048,
                                                     'input_size': 512,
                                                     'nhead': 8,
                                                     'num_decoder_layers': 0,
                                                     'num_encoder_layers': 6,
                                                     'output_size': 512,
                                                     'tfm_activation': 'relu',
                                                     'tfm_dropout': 0.1}),
 'Ignore_PSN_Layers': ['B', 'P'],
 'embedprocess': {'dropout': {'inplace': False, 'p': 0.5},
                  'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},
 'input_size': 512,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
                 'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},


# Reducer

## Module

In [43]:
import torch

# from ..nn.op import ReduceMeanLayer, ReduceSumLayer, RecuderMaxLayer, ConcatenateLayer
from fieldnn.nn.op import ReduceMeanLayer, ReduceSumLayer, RecuderMaxLayer, ConcatenateLayer

class Reducer_Layer(torch.nn.Module):
    def __init__(self, input_fullname, output_fullname, reducer_layer_para):
        super(Reducer_Layer, self).__init__()
        # (bs, xxx, l, c_inp) --> (bs, xxx, c_outp)
        
        # Part 0: Meta
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        self.input_size = reducer_layer_para['input_size']
        self.output_size = reducer_layer_para['output_size']

        # Part 1: NN
        nn_name, nn_para = reducer_layer_para[input_fullname]
        if nn_name.lower() == 'mean':
            self.reducer = ReduceMeanLayer()
        elif nn_name.lower() == 'sum':
            self.reducer = ReduceSumLayer()
        elif nn_name.lower() == 'max':
            self.reducer = RecuderMaxLayer()
        elif nn_name.lower() == 'concat':
            self.reducer = ConcatenateLayer()
            # TODO: need to assert something
            assert self.output_size % self.input_size == 0
        else:
            raise ValueError(f'There is no layer for "{nn_name}"')
            
        # Part 3: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in reducer_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
            
    def forward(self, fullname, holder, info):
        assert self.input_fullname == fullname
        leng_mask = holder == 0
        info = self.reducer(info, leng_mask)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)
            
        # fullname = self.fullname
        holder = (leng_mask == 0).sum(-1)
        return self.output_fullname, holder, info

In [44]:
# from fieldnn.sublayer.reducer import Reducer_Layer

## Config

In [45]:
# def get_reducer_para(fullname, nn_name, nn_para, input_size, output_size, postprocess):
#     reducer_layer_para = {}
    
#     reducer_layer_para[fullname] = nn_name, nn_para
    
#     reducer_layer_para['input_size'] = input_size
#     reducer_layer_para['output_size'] = output_size

#     reducer_layer_para['postprocess'] = postprocess
    
#     return reducer_layer_para

In [46]:
from fieldnn.utils.parafn import get_reducer_para

## Usage

### Expander

In [47]:
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 100

Ignore_PSN_Layers = ['B', 'P']

fullname = 'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn'
# fld_name = 'B2St2TkGrn'
info_idx = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(info_idx.shape)
holder = torch.LongTensor(info_idx)

from pprint import pprint
# embed_size = 512
vocab_size = 5001
process = {
    'dropout' :{'p':0.5, 'inplace':False},
    'layernorm': {'eps': 1e-05, "elementwise_affine":True},
}


###########
nn_name = 'Embedding'
nn_para = {'input_size': vocab_size}
postprocess = process
###########

input_fullname = fullname
output_fullname = input_fullname.replace('Grn', '')

expander_layer_para = get_expander_para(fullname, nn_name, nn_para, embed_size, 
                                        Ignore_PSN_Layers, 
                                        postprocess
                                       )
pprint(expander_layer_para)
Expander = Expander_Layer(input_fullname, output_fullname, expander_layer_para)
print(Expander)


info = 'None'
fullname, holder, info = Expander(fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)



###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########
input_fullname = output_fullname
output_fullname = input_fullname
learner_layer_para  = get_learner_para(fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
pprint(learner_layer_para)

Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)
# print(Learner)

fullname, holder, info = Learner(fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)


1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 7)
3
3 --> (3, 6, 7)
4 --> (3, 6, 7, 4)
(3, 6, 7, 4)
{'B-P-EC:Diag-Rec:DiagV-DiagVdftGrn': ('Embedding',
                                       {'embedding_size': 512,
                                        'init': 'random',
                                        'input_size': 5001}),
 'Ignore_PSN_Layers': ['B', 'P'],
 'input_size': None,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
                 'layernorm': {'elementwise_affine': True, 'eps': 1e-05}}}
Expander_Layer(
  (Embed): EmbeddingLayer(
    (embedding): Embedding(5001, 512, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)
torch.Size([3, 6, 7, 4, 512])
B-P-EC:Diag-Rec:DiagV-DiagVdft
torch.Size([3, 6, 7, 4])
{'B-P-EC:Diag-Rec:DiagV-DiagVdft': ('TFM',
                                    {'dim_feedforward': 2048,
       

### Merger

In [48]:
from fieldnn.utils.parafn import get_fullname_from_inputs


fullnamea = 'B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA'
fullnameb = 'B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftB'

fullname_list = [fullnamea, fullnameb]
fullname2data = {i: {'holder': holder, 'info': info} for i in fullname_list}

input_fullname = '^'.join(fullname_list)
print(input_fullname)
output_fullname = get_fullname_from_inputs(fullname_list)
print(output_fullname)


for fn_input, data in fullname2data.items():
    print(fn_input, data['holder'].shape, data['info'].shape)
print(fullname_list)


#########
nn_name = 'Merger'
nn_para = {}
input_size = embed_size
output_size = embed_size
postprocess = process
#########

merger_layer_para = get_merger_para(input_fullname, nn_name, nn_para, input_size, output_size, postprocess)
pprint(merger_layer_para)

Merger = Merger_Layer(input_fullname, output_fullname, merger_layer_para)
print(Merger)


output_fullname, holder, info = Merger(input_fullname, fullname2data)
print(holder.shape)
print(info.shape)
print(output_fullname)

B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA^B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftB
B-P-ECRec:Diag-DiagRec:DiagV-VRec-DiagVdftA&DiagVdftB
B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA torch.Size([3, 6, 7, 4]) torch.Size([3, 6, 7, 4, 512])
B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftB torch.Size([3, 6, 7, 4]) torch.Size([3, 6, 7, 4, 512])
['B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA', 'B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftB']
{'B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA^B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftB': ('Merger',
                                                                                             {}),
 'input_size': 512,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
                 'layernorm': {'elementwise_affine': True, 'eps': 1e-05}}}
Merger_Layer(
  (Merger): MergerLayer()
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [49]:
output_fullname

'B-P-ECRec:Diag-DiagRec:DiagV-VRec-DiagVdftA&DiagVdftB'

In [50]:
###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########
input_fullname = output_fullname
output_fullname = input_fullname
learner_layer_para  = get_learner_para(input_fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
pprint(learner_layer_para)

Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)
# print(Learner)

fullname, holder, info = Learner(input_fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)

{'B-P-ECRec:Diag-DiagRec:DiagV-VRec-DiagVdftA&DiagVdftB': ('TFM',
                                                           {'dim_feedforward': 2048,
                                                            'input_size': 512,
                                                            'nhead': 8,
                                                            'num_decoder_layers': 0,
                                                            'num_encoder_layers': 6,
                                                            'output_size': 512,
                                                            'tfm_activation': 'relu',
                                                            'tfm_dropout': 0.1}),
 'Ignore_PSN_Layers': ['B', 'P'],
 'embedprocess': {'dropout': {'inplace': False, 'p': 0.5},
                  'layernorm': {'elementwise_affine': True, 'eps': 1e-05}},
 'input_size': 512,
 'output_size': 512,
 'postprocess': {'dropout': {'inplace': False, 'p': 0.5},
            

### First Reduce

In [51]:
print(fullname)

B-P-ECRec:Diag-DiagRec:DiagV-VRec-DiagVdftA&DiagVdftB


In [52]:
input_fullname = fullname
output_fullname = '-'.join(fullname.split('-')[:-1]) + ':' + fullname.split('-')[- 1]
# fields = input_fullname.split('-')[-1].split('&')
print(input_fullname)
print(output_fullname)
# print(fields)

B-P-ECRec:Diag-DiagRec:DiagV-VRec-DiagVdftA&DiagVdftB
B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA&DiagVdftB


In [53]:
#########
nn_name = 'Max'
nn_para = {}
input_size = embed_size
output_size = embed_size if nn_name != 'concat' else embed_size * 3
postprocess = process
#########

reducer_layer_para = get_reducer_para(input_fullname, nn_name, nn_para, input_size, output_size, postprocess)
print(reducer_layer_para)
Reducer = Reducer_Layer(input_fullname, output_fullname, reducer_layer_para)
Reducer

{'B-P-ECRec:Diag-DiagRec:DiagV-VRec-DiagVdftA&DiagVdftB': ('Max', {}), 'input_size': 512, 'output_size': 512, 'postprocess': {'dropout': {'p': 0.5, 'inplace': False}, 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}


Reducer_Layer(
  (reducer): RecuderMaxLayer()
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [54]:
output_fullname, holder_new, info_new = Reducer(input_fullname, holder, info)
print(output_fullname)
print(holder_new.shape)
print(info_new.shape)

B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA&DiagVdftB
torch.Size([3, 6, 7, 4])
torch.Size([3, 6, 7, 4, 512])


In [55]:
input_fullname = output_fullname
output_fullname = input_fullname
holder = holder_new
info = info_new


###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########
learner_layer_para  = get_learner_para(input_fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
print(fullname)
print(learner_layer_para)
Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)


fullname, holder, info = Learner(input_fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)

B-P-ECRec:Diag-DiagRec:DiagV-VRec-DiagVdftA&DiagVdftB
{'input_size': 512, 'output_size': 512, 'B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA&DiagVdftB': ('TFM', {'input_size': 512, 'output_size': 512, 'nhead': 8, 'num_encoder_layers': 6, 'num_decoder_layers': 0, 'dim_feedforward': 2048, 'tfm_dropout': 0.1, 'tfm_activation': 'relu'}), 'psn_layers': ['VRec:DiagVdftA&DiagVdftB', 'DiagRec:DiagV', 'ECRec:Diag'], 'embedprocess': {'dropout': {'p': 0.5, 'inplace': False}, 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}, 'Ignore_PSN_Layers': ['B', 'P'], 'postprocess': {'dropout': {'p': 0.5, 'inplace': False}, 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}
torch.Size([3, 6, 7, 4, 512])
B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA&DiagVdftB
torch.Size([3, 6, 7, 4])


### Second Reduce

In [56]:
fullname

'B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA&DiagVdftB'

In [57]:
input_fullname = fullname
output_fullname = '-'.join(fullname.split('-')[:-1]) + ':' + fullname.split('-')[- 1]
# fields = fullname_input.split('-')[-1].split('&')
print(input_fullname)
print(output_fullname)
# print(fields)

B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA&DiagVdftB
B-P-ECRec:Diag-DiagRec:DiagV:VRec:DiagVdftA&DiagVdftB


In [58]:
#########
nn_name = 'Max'
nn_para = {}
input_size = embed_size
output_size = embed_size if nn_name != 'concat' else embed_size * 3
postprocess = process
#########

reducer_layer_para = get_reducer_para(input_fullname, nn_name, nn_para, input_size, output_size, postprocess)
print(reducer_layer_para)
Reducer = Reducer_Layer(input_fullname, output_fullname, reducer_layer_para)
Reducer

{'B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA&DiagVdftB': ('Max', {}), 'input_size': 512, 'output_size': 512, 'postprocess': {'dropout': {'p': 0.5, 'inplace': False}, 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}


Reducer_Layer(
  (reducer): RecuderMaxLayer()
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [59]:
output_fullname, holder_new, info_new = Reducer(input_fullname, holder, info)
print(output_fullname)
print(holder_new.shape)
print(info_new.shape)

B-P-ECRec:Diag-DiagRec:DiagV:VRec:DiagVdftA&DiagVdftB
torch.Size([3, 6, 7])
torch.Size([3, 6, 7, 512])


In [60]:
input_fullname = output_fullname
output_fullname = input_fullname
holder = holder_new
info = info_new


###########
nn_name = 'TFM'
nn_para = {'num_encoder_layers': 6}
input_size = embed_size
output_size = embed_size
embedprocess = process
postprocess = process
###########
learner_layer_para  = get_learner_para(input_fullname, 
                                       nn_name, nn_para, 
                                       input_size, output_size, 
                                       Ignore_PSN_Layers, 
                                       embedprocess, postprocess
                                      )
print(fullname)
print(learner_layer_para)
Learner = Learner_Layer(input_fullname, output_fullname, learner_layer_para)


fullname, holder, info = Learner(input_fullname, holder, info)
print(info.shape)
print(fullname)
print(holder.shape)

B-P-ECRec:Diag-DiagRec:DiagV-VRec:DiagVdftA&DiagVdftB
{'input_size': 512, 'output_size': 512, 'B-P-ECRec:Diag-DiagRec:DiagV:VRec:DiagVdftA&DiagVdftB': ('TFM', {'input_size': 512, 'output_size': 512, 'nhead': 8, 'num_encoder_layers': 6, 'num_decoder_layers': 0, 'dim_feedforward': 2048, 'tfm_dropout': 0.1, 'tfm_activation': 'relu'}), 'psn_layers': ['DiagRec:DiagV:VRec:DiagVdftA&DiagVdftB', 'ECRec:Diag'], 'embedprocess': {'dropout': {'p': 0.5, 'inplace': False}, 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}, 'Ignore_PSN_Layers': ['B', 'P'], 'postprocess': {'dropout': {'p': 0.5, 'inplace': False}, 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}
torch.Size([3, 6, 7, 512])
B-P-ECRec:Diag-DiagRec:DiagV:VRec:DiagVdftA&DiagVdftB
torch.Size([3, 6, 7])
